# Финальное задание

# Подход 1: Градиентный бустинг

### Шаг 1:

In [49]:
import pandas as pd

df = pd.read_csv('./features.csv', index_col='match_id')
# print (df.head())
cols_num = len(df.columns)
cols_to_drop = list(df.columns.values[(cols_num - 6):])
print(f'Признаки, связанные с итогами матча: {cols_to_drop}')
y = df['radiant_win'] #целевая переменная
df = df.drop(columns = cols_to_drop)
# print (df.head())

Признаки, связанные с итогами матча: ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire']


### Шаг 2:

In [46]:
def missed_labels(df):
    count_series = df.count()
    labels = []
    max_val = max(count_series)
    for i in range(0, len(count_series)):
        ind = count_series.index[i] 
        if count_series[ind] != max_val:
            labels.append(ind)
    return labels
            
print (f'Признаки с пропущенными значениями: {missed_labels(df)}')

Признаки с пропущенными значениями: ['first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time']


**Какие признаки имеют пропуски среди своих значений (приведите полный список имен этих признаков)? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?**

Все, что относится к `first_blood_*` легко объясняется тем, что если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение.
Пропущенные значения для `radiant_*` и `dire_*`: время приобретения (для `first_ward_time` время установки) предметов обеими командами. Данные события могут не произойти в первые 5 минут игры.

### Шаг 3:

In [47]:
def subst_missed(mode, df):
    if mode == 'zeros':
        df_out = df.fillna(0)
    elif mode == 'min':
        df_out = df.fillna(df.min())
    elif mode == 'max':
        df_out = df.fillna(df.max())
    elif mode == 'mean':
        df_out = df.fillna(df.mean())
    return df_out
        
mode = ['zeros', 'min', 'max', 'mean']#способы заполнения пропущенных значений: нулями, минимумом, максимумом, средним
# print (f'before:{missed_labels(df)}')
df_wout_miss = subst_missed(mode[0], df)
# print (f'after:{missed_labels(df_wout_miss)}')

### Шаг 4: 
**Как называется столбец, содержащий целевую переменную?** 

целевая переменная - `radiant_win` 1, если победила команда Radiant, 0 — иначе

### Шаг 5:

In [139]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn import utils
import time
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

**Обработка тестовой выборки:**

In [62]:
df_test = pd.read_csv('./features_test.csv', index_col='match_id')
df_test_wout_miss = subst_missed(mode[0], df_test)
X_test = df_test_wout_miss.as_matrix()


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
y_train = y.as_matrix()
X_train = df_wout_miss.as_matrix()

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [141]:
def cvKfold(X,y):
    kf = KFold(n_splits=5, random_state=1, shuffle=True)
    kf.get_n_splits(X, y)
    return kf


In [168]:
def GB_clf_new(X_train, y_train, trees):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(learning_rate = 0.1, n_estimators=trees, verbose=True, random_state=241, max_depth = 3)
    kf = cvKfold(X_train, y_train)
    start_time = datetime.datetime.now()
    cvs = cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=kf, n_jobs=-1)
    cvs_val = cvs.mean()
    print (f'esimators: {trees}\ncross-validation score: {cvs_val}')
    print (f'Time elapsed:{datetime.datetime.now() - start_time}')

    

In [169]:
trees_num = [10, 20, 30, 40, 50, 60, 80, 100, 200, 250, 400]
for tr in trees_num:
    GB_clf_new(X_train, y_train, tr)


esimators: 10
cross-validation score: 0.663027010030855
Time elapsed:0:00:31.680822
esimators: 20
cross-validation score: 0.681569552464358
Time elapsed:0:00:54.092849
esimators: 30
cross-validation score: 0.6891385982606287
Time elapsed:0:01:16.505197
esimators: 40
cross-validation score: 0.6936068144820396
Time elapsed:0:01:47.376102
esimators: 50
cross-validation score: 0.6969531965219121
Time elapsed:0:02:09.152996
esimators: 60
cross-validation score: 0.6996693178695128
Time elapsed:0:02:34.847590
esimators: 80
cross-validation score: 0.7033894534658846
Time elapsed:0:03:31.034234
esimators: 100
cross-validation score: 0.7059011910007141
Time elapsed:0:04:05.687396
esimators: 200
cross-validation score: 0.7131874038983415
Time elapsed:0:07:55.500213
esimators: 250
cross-validation score: 0.7153974121383734
Time elapsed:0:09:50.045518
esimators: 400
cross-validation score: 0.7191431330297183
Time elapsed:0:15:36.489409


**Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями?**
Time elapsed:0:01:16.505197

**Какое качество при этом получилось?**
cross-validation score: 0.6891385982606287

**Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что можно сделать, чтобы ускорить его обучение при увеличении количества деревьев?**

Использовать большее количество деревьев имеет смысл, чтобы увеличить точность модели. Нужно учитывать, что с увеличением количества деревьев, увеличивается не только оценка качества, но и время обучения. Чтобы уменьшить время обучения можно использовать часть выборки, а также уменьшить глубину деревьев (при `max_depth = 1` время выполнения сокращается примерно в 3 раза).

*Проверим другие способы заполнения пропущенных значений для 10 деревьев:*

In [170]:
for m in mode:
    df_wout_miss = subst_missed(m, df)
    X_train = df_wout_miss.as_matrix()
    print (f'{m}')
    GB_clf_new(X_train, y_train, 10)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


zeros
esimators: 10
cross-validation score: 0.66483292280491
Time elapsed:0:00:31.144076
min
esimators: 10
cross-validation score: 0.66483292280491
Time elapsed:0:00:27.499708
max
esimators: 10
cross-validation score: 0.6668857850212715
Time elapsed:0:00:26.425586
mean
esimators: 10
cross-validation score: 0.663027010030855
Time elapsed:0:00:26.891059


Для данной задачи заполнение максимальным значением показывает лучший результат с точки зрения оценки качества

# Подход 2: Логистическая регрессия

In [171]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

In [244]:
def LR(X_train, y_train, C):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    regr = LogisticRegression(C = C,  random_state=241, n_jobs=-1)
    kf = cvKfold(X_train, y_train)
    start_time = datetime.datetime.now()
    cvs = cross_val_score(regr, X_train, y_train, scoring='roc_auc', cv=kf, n_jobs=-1)
    cvs_val = cvs.mean()
    print (f'C_regularization: {C}\ncross-validation score: {cvs_val}')
    print (f'Time elapsed:{datetime.datetime.now() - start_time}')


### Шаг 1:

In [175]:
C_values = np.power(10.0, np.arange(-5, 6))

for C in C_values:
    LR(X_train, y_train, C)

C_regularization: 1e-05
cross-validation score: 0.6945298784714038
Time elapsed:0:00:02.720724
C_regularization: 0.0001
cross-validation score: 0.7120012896775183
Time elapsed:0:00:03.439803
C_regularization: 0.001
cross-validation score: 0.7166125177369221
Time elapsed:0:00:06.045833
C_regularization: 0.01
cross-validation score: 0.7167538783462648
Time elapsed:0:00:07.952547
C_regularization: 0.1
cross-validation score: 0.7167265024959126
Time elapsed:0:00:08.167459
C_regularization: 1.0
cross-validation score: 0.7167226184902952
Time elapsed:0:00:08.314383
C_regularization: 10.0
cross-validation score: 0.7167225358878273
Time elapsed:0:00:08.878269
C_regularization: 100.0
cross-validation score: 0.7167224426942234
Time elapsed:0:00:08.608596
C_regularization: 1000.0
cross-validation score: 0.7167224300026743
Time elapsed:0:00:08.589241
C_regularization: 10000.0
cross-validation score: 0.7167224278889275
Time elapsed:0:00:08.336592
C_regularization: 100000.0
cross-validation score: 0

**Какое наилучшее качество у вас получилось?**
C_regularization: 0.1
cross-validation score: 0.7167265024959126

**Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?**
Данное значение немного меньше, чем оценка качества градиентного бустинга с 400 деревьями (0.719), но логистическая регрессия обучается намного быстрее по сравнению с градиентным бустингом: 8 секунд vs 15 минут. Линейные методы работают быстрее в вижу своей простоты.


### Шаг 2:

In [234]:
df = pd.read_csv('./features.csv', index_col='match_id')
cols_num = len(df.columns)
cols_to_drop = list(df.columns.values[(cols_num - 6):])
df = df.drop(columns = cols_to_drop)
cols_category = ['lobby_type']
for p in range(0,5):
    cols_category.append('r%d_hero' % (p+1))
    cols_category.append('d%d_hero' % (p+1))
print(f'Категориальные признаки: {cols_category}')
df_heroes = df.loc[:,cols_category[1:]]
df_1 = df.drop(columns = cols_category)
df_wout_miss = subst_missed(mode[0], df_1)

Категориальные признаки: ['lobby_type', 'r1_hero', 'd1_hero', 'r2_hero', 'd2_hero', 'r3_hero', 'd3_hero', 'r4_hero', 'd4_hero', 'r5_hero', 'd5_hero']


In [186]:
df_wout_miss = subst_missed(mode[0], df)
X_train = df_wout_miss.as_matrix()

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [187]:
for C in C_values:
    LR(X_train, y_train, C)

C_regularization: 1e-05
cross-validation score: 0.6951110694353423
Time elapsed:0:00:06.197483
C_regularization: 0.0001
cross-validation score: 0.7112629500700418
Time elapsed:0:00:04.748302
C_regularization: 0.001
cross-validation score: 0.7162333103715236
Time elapsed:0:00:05.657995
C_regularization: 0.01
cross-validation score: 0.7164088702736156
Time elapsed:0:00:07.872319
C_regularization: 0.1
cross-validation score: 0.716383801538205
Time elapsed:0:00:08.024546
C_regularization: 1.0
cross-validation score: 0.7163803390297804
Time elapsed:0:00:08.645960
C_regularization: 10.0
cross-validation score: 0.7163798497335823
Time elapsed:0:00:08.022548
C_regularization: 100.0
cross-validation score: 0.7163796823834381
Time elapsed:0:00:07.804136
C_regularization: 1000.0
cross-validation score: 0.7163796929771316
Time elapsed:0:00:07.623997
C_regularization: 10000.0
cross-validation score: 0.716379699336385
Time elapsed:0:00:07.737887
C_regularization: 100000.0
cross-validation score: 0.7

**Изменилось ли качество? Чем вы можете это объяснить?**

После удаления категориальных признаков качество практически не изменилось, местами ухудшилось. Некоторых пресонажей выбирают чаще, так как на них легче играть или они сильнее, а значит побед на таких персонажах будет больше.

### Шаг 3:

**Сколько различных идентификаторов героев существует в данной игре?**

In [218]:
heroes = df_heroes.as_matrix()
N = len(np.unique(heroes))
N_max = max(np.unique(heroes))
print(f'{N} различных идентификаторов героев')


108 различных идентификаторов героев


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


### Шаг 4

In [240]:
X_pick = np.zeros((df_heroes.shape[0], N_max))

for i, match_id in enumerate(df.index):
    for p in range(0,5):
        X_pick[i, df.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, df.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
heroes_df = pd.DataFrame(X_pick, index=df.index)
X_train_all = pd.concat([df_wout_miss, heroes_df], axis=1)

### Шаг 5:

In [241]:
X_train = X_train_all.as_matrix()

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [245]:
for C in C_values:
    LR(X_train, y_train, C)

C_regularization: 1e-05
cross-validation score: 0.7148913479970942
Time elapsed:0:00:04.345382
C_regularization: 0.0001
cross-validation score: 0.7428571510322293
Time elapsed:0:00:06.205409
C_regularization: 0.001
cross-validation score: 0.7516014700448687
Time elapsed:0:00:11.103352
C_regularization: 0.01
cross-validation score: 0.7519047305979785
Time elapsed:0:00:14.954012
C_regularization: 0.1
cross-validation score: 0.7518617237159408
Time elapsed:0:00:15.197360
C_regularization: 1.0
cross-validation score: 0.7518560551551905
Time elapsed:0:00:15.173422
C_regularization: 10.0
cross-validation score: 0.7518554325401261
Time elapsed:0:00:15.277147
C_regularization: 100.0
cross-validation score: 0.7518553435406548
Time elapsed:0:00:15.392176
C_regularization: 1000.0
cross-validation score: 0.7518553308360634
Time elapsed:0:00:15.182400
C_regularization: 10000.0
cross-validation score: 0.7518553265919965
Time elapsed:0:00:15.363075
C_regularization: 100000.0
cross-validation score: 0

**Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?**

Качество заметно улучшилось, это объясняется тем, что была проведена предобработка данных и преобразованные категориальные признаки дают больше информации, чем "сырые".

### Шаг 6:

In [259]:
def LR_best(X_train, y_train, X_test):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    regr = LogisticRegression(C = 0.01,  random_state=241, n_jobs=-1)
    regr.fit(X_train, y_train)
    y_pred = regr.predict_proba(X_test)
    return y_pred


In [265]:
def preprocessing_df(df):
    cols_num = len(df.columns)
#     cols_to_drop = list(df.columns.values[(cols_num - 6):])
#     df = df.drop(columns = cols_to_drop)
    cols_category = ['lobby_type']
    for p in range(0,5):
        cols_category.append('r%d_hero' % (p+1))
        cols_category.append('d%d_hero' % (p+1))
    df_heroes = df.loc[:,cols_category[1:]]
    df_1 = df.drop(columns = cols_category)
    df_wout_miss = subst_missed(mode[0], df_1)
    heroes = df_heroes.as_matrix()
    N = len(np.unique(heroes))
    N_max = max(np.unique(heroes))
    X_pick = np.zeros((df_heroes.shape[0], N_max))

    for i, match_id in enumerate(df.index):
        for p in range(0,5):
            X_pick[i, df.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, df.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
        
    heroes_df = pd.DataFrame(X_pick, index=df.index)
    X_train_all = pd.concat([df_wout_miss, heroes_df], axis=1)
    #print(X_train_all.head())
    X_train = X_train_all.as_matrix()
    return X_train

In [266]:
df_test = pd.read_csv('./features_test.csv', index_col='match_id')
X_test = preprocessing_df(df_test)


C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [268]:
pred = LR_best(X_train, y_train, X_test)

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1297: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


**Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?**

In [274]:
print (f'min: {pred.min()}, max: {pred.max()}')

min: 0.0036712840745719566, max: 0.996328715925428
